# 2110010114 李佳琪 第四次作业

1. 查找数据集中的缺失值；
2. 用该特征的特征均值进行缺失值填补；
3. 使用三倍标准差法处理每个特征中的异常值；
4. 计算信用数据集中四个特征‘总贷款量’,‘贷款利率’,‘总还款量’和‘上一次还款量’在两种信用状态(loan_status)下的均值、方差、中位数；
5. 计算违约状态(loan_status)两种状态的样本比例，并采用随机上采样或随机下采样平衡数据集；
6. 计算平衡之后数据集中四个特征‘总贷款量’,‘贷款利率’,‘总还款量’和‘上一次还款量’在两种loan_status下的均值、方差、中位数。

In [16]:
import pandas as pd  
import numpy as np  
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [17]:
# 读取Excel文件  
df = pd.read_excel('D:\\大三上\\数据挖掘与商务智能\\第四次实验课20231030\信用数据集.xlsx')     

# 查找缺失值
missing_values = df.isnull().sum()
print (missing_values)

总贷款量                1
贷款利率                2
总还款量                2
上一次还款量              1
违约状态（0-违约，1-未违约）    0
dtype: int64


In [18]:
# 计算特征均值
mean_values = df.mean()
print (mean_values)

总贷款量                13424.665179
贷款利率                    0.134941
总还款量                10069.240602
上一次还款量               2310.042812
违约状态（0-违约，1-未违约）        0.777778
dtype: float64


In [19]:
# 填充缺失值
df.fillna(mean_values, inplace=True)

# 使用三倍标准差法处理异常值
for column in df.columns:
    mean = df[column].mean()
    std = df[column].std()
    threshold = 3 * std
    df[column] = np.where((df[column] - mean).abs() > threshold, mean, df[column])
print (df)

        总贷款量    贷款利率          总还款量       上一次还款量  违约状态（0-违约，1-未违约）
0     2500.0  0.1527   1008.710000   119.660000               0.0
1     5600.0  0.2128    646.020000   152.390000               0.0
2     5375.0  0.1269   1476.190000   121.450000               0.0
3     9000.0  0.1349   2270.700000   305.380000               0.0
4    10000.0  0.1065   7471.990000   325.740000               0.0
..       ...     ...           ...          ...               ...
220   8000.0  0.0579   6297.940000  1110.770000               1.0
221  15000.0  0.1905  19729.506580  2680.340000               1.0
222   8125.0  0.1249   3521.860000   271.780000               1.0
223  30000.0  0.1849  10069.240602  2310.042812               1.0
224  12000.0  0.1212  14373.353090   399.250000               1.0

[225 rows x 5 columns]


In [20]:
# 选择感兴趣的特征
features = ['总贷款量', '贷款利率', '总还款量', '上一次还款量']

In [21]:
# 计算违约时的统计指标
status_0_stats = df[df['违约状态（0-违约，1-未违约）'] == 0][features].agg(['mean', 'var', 'median'])

# 计算不违约时的统计指标
status_1_stats = df[df['违约状态（0-违约，1-未违约）'] == 1][features].agg(['mean', 'var', 'median'])

# 打印结果
print("违约状态的统计指标：")
print(status_0_stats)
print("未违约状态的统计指标：")
print(status_1_stats)


违约状态的统计指标：
                总贷款量      贷款利率          总还款量         上一次还款量
mean    1.215249e+04  0.138399  7.600689e+03     422.144200
var     3.860259e+07  0.001244  2.792443e+07  379170.618706
median  1.080000e+04  0.134900  6.642795e+03     322.210000
未违约状态的统计指标：
                总贷款量      贷款利率          总还款量        上一次还款量
mean    1.378814e+04  0.133953  1.001363e+04  1.825708e+03
var     6.278455e+07  0.001748  4.572063e+07  9.826564e+06
median  1.200000e+04  0.129900  8.314440e+03  4.981500e+02


In [22]:
# 计算违约状态为0或1的样本比例
status_0_ratio = (df['违约状态（0-违约，1-未违约）'] == 0).sum() / len(df)
status_1_ratio = (df['违约状态（0-违约，1-未违约）'] == 1).sum() / len(df)

# 打印原始数据集中状态0和1的样本比例
print("原始数据集中违约状态的样本比例:", status_0_ratio)
print("原始数据集中未违约状态的样本比例:", status_1_ratio)

原始数据集中违约状态的样本比例: 0.2222222222222222
原始数据集中未违约状态的样本比例: 0.7777777777777778


In [23]:
# 增加下采样比例，例如将比例设置为1.0（完全平衡）
under_sampler = RandomUnderSampler(sampling_strategy=1.0)

# 执行下采样
X_resampled, y_resampled = under_sampler.fit_resample(df[features], df['违约状态（0-违约，1-未违约）'])

# 创建平衡后的数据集DataFrame
balanced_df = pd.DataFrame(X_resampled, columns=features)
balanced_df['违约状态（0-违约，1-未违约）'] = y_resampled

In [24]:
# 计算平衡后数据集中状态=0和1的均值、方差和中位数
status_0_stats = balanced_df[balanced_df['违约状态（0-违约，1-未违约）'] == 0][features].agg(['mean', 'var', 'median'])
status_1_stats = balanced_df[balanced_df['违约状态（0-违约，1-未违约）'] == 1][features].agg(['mean', 'var', 'median'])

# 打印结果
print("\n平衡后数据集中违约状态的统计指标：")
print(status_0_stats)
print("\n平衡后数据集中不违约的统计指标：")
print(status_1_stats)


平衡后数据集中违约状态的统计指标：
                总贷款量      贷款利率          总还款量         上一次还款量
mean    1.215249e+04  0.138399  7.600689e+03     422.144200
var     3.860259e+07  0.001244  2.792443e+07  379170.618706
median  1.080000e+04  0.134900  6.642795e+03     322.210000

平衡后数据集中不违约的统计指标：
                总贷款量      贷款利率          总还款量        上一次还款量
mean    1.259900e+04  0.136894  1.013660e+04  2.501734e+03
var     6.238184e+07  0.001516  4.465599e+07  1.650743e+07
median  1.000000e+04  0.124400  8.281740e+03  5.161100e+02
